<a href="https://colab.research.google.com/github/HesusG/diagnostico-lineas-accion/blob/main/Semana2/ejercicios_extra/practica_ong_estadistica_avanzada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏥 Práctica Extra: Fundación Esperanza - Estadística Avanzada

**Curso:** CD2001B - Diagnóstico para Líneas de Acción  
**Módulo:** 2 - Estadística Avanzada para Negocios  
**Tipo:** Ejercicio de práctica adicional  
**Dataset:** fundacion_esperanza_donadores.csv  
**Tiempo estimado:** 120-150 minutos

---

## 🎯 Objetivo de Aprendizaje

Aplicar técnicas de **estadística avanzada** para analizar estrategias de retención de donadores, incluyendo:

- **Chi-cuadrada (χ²):** Analizar independencia entre variables categóricas
- **ANOVA:** Comparar medias entre múltiples grupos
- **Regresión Lineal:** Predecir variables continuas
- **Reflexión MEAL Comparativa:** Integrar hallazgos de Semanas 1 y 2

---

## 📖 Contexto del Caso

### Continuación del Análisis - Fundación Esperanza

En la **práctica de Semana 1**, investigaste:
- Satisfacción general y confianza financiera (descriptivos)
- Diferencias entre voluntarios vs no voluntarios (prueba t)
- Diferencias entre retenidos vs no retenidos (prueba t)

**Hallazgos clave de Semana 1:**
- Tasa de retención: ~40% (área de mejora)
- Voluntarios tienen mayor conexión emocional
- Donadores retenidos tienen mayor satisfacción

---

### Nuevas Preguntas de Negocio - Semana 2

El Departamento de Desarrollo quiere profundizar con preguntas más complejas:

1. **¿El tipo de donador está relacionado con la frecuencia de donación?** (Chi-cuadrada)
   - ¿Donadores corporativos donan más frecuentemente que individuales?
   - ¿Hay patrones de comportamiento por segmento?

2. **¿La satisfacción varía según el canal de donación?** (ANOVA)
   - ¿Qué canal genera mayor satisfacción: online, eventos, presencial?
   - ¿Deberíamos priorizar ciertos canales?

3. **¿Los años como donador predicen el compromiso futuro?** (Regresión)
   - ¿La antigüedad genera mayor lealtad?
   - ¿Podemos predecir compromiso basándonos en la trayectoria?

---

## 📊 Variables Relevantes para Esta Práctica

**Variables categóricas:**
- `tipo_donante`: Individual, Corporativo, Fundación, Primera vez
- `frecuencia_donacion`: Mensual, Trimestral, Anual, Ocasional
- `canal_donacion`: Online, Evento, Presencial, Teléfono

**Variables numéricas:**
- `años_como_donante`: Años de relación con la Fundación (0-20)
- `satisfaccion_general`: Escala Likert 1-10
- `compromiso_futuro`: Escala Likert 1-10

📚 **Documentación completa:** `CODEBOOK_fundacion_esperanza.md`

---

## Parte 0: Configuración y Carga de Datos (5 puntos)

### 0.1 Importar Librerías

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency, f_oneway
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualizaciones
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
%matplotlib inline

print("✅ Librerías importadas exitosamente")

### 0.2 Cargar Dataset

In [ ]:
# Cargar dataset
# Opción 1: Local (si descargaste el repositorio)
df = pd.read_csv('https://raw.githubusercontent.com/HesusG/diagnostico-lineas-accion/main/Semana1/datos/fundacion_esperanza_donadores.csv')

# Opción 2: Desde GitHub (si estás en Colab)
# df = pd.read_csv('https://raw.githubusercontent.com/HesusG/diagnostico-lineas-accion/main/Semana1/datos/fundacion_esperanza_donadores.csv')

print(f"📊 Dataset cargado: {df.shape[0]} donadores × {df.shape[1]} variables")
print(f"\n🔍 Primeras 5 filas:")
df.head()

### 0.3 Inspección de Variables Relevantes

In [ ]:
# Variables que usaremos en esta práctica
vars_interes = ['tipo_donante', 'frecuencia_donacion', 'canal_donacion', 
                'años_como_donante', 'satisfaccion_general', 'compromiso_futuro']

print("📋 VARIABLES DE INTERÉS:\n")
print(df[vars_interes].info())

print("\n" + "="*70)
print("❓ Verificar valores faltantes:")
print(df[vars_interes].isnull().sum())

print("\n" + "="*70)
print("🔢 Estadísticas descriptivas (variables numéricas):")
print(df[['años_como_donante', 'satisfaccion_general', 'compromiso_futuro']].describe())

---

## Parte 1: Prueba Chi-Cuadrada (χ²) de Independencia (25 puntos)

### Contexto

**Pregunta de negocio:** ¿El tipo de donador (`tipo_donante`) está relacionado con la frecuencia de donación (`frecuencia_donacion`)?

**Hipótesis intuitiva:**
- Donadores **corporativos** podrían donar más frecuentemente (presupuestos anuales)
- Donadores **individuales** podrían tener patrones más variables
- Donadores de **primera vez** probablemente están en categoría "Ocasional"

**¿Por qué Chi-cuadrada?** Ambas variables son categóricas.

---

### 1.1 Planteamiento de Hipótesis (5 puntos)

**📝 Ejercicio 1.1:** Escribe las hipótesis estadísticas.

**Hipótesis:**

- **H₀ (Nula):** [TU RESPUESTA - ¿Las variables son independientes?]

- **H₁ (Alternativa):** [TU RESPUESTA - ¿Existe asociación entre las variables?]

**Nivel de significancia:** α = 0.05

### 1.2 Tabla de Contingencia (5 puntos)

In [ ]:
# TU CÓDIGO AQUÍ: Crea tabla de contingencia
# Sugerencia: pd.crosstab(df['tipo_donante'], df['frecuencia_donacion'])

tabla_contingencia = 

print("📊 TABLA DE CONTINGENCIA: Tipo de Donante × Frecuencia de Donación")
print("="*70)
print(tabla_contingencia)
print(f"\nTotal de donadores: {tabla_contingencia.sum().sum()}")

In [ ]:
# Visualización: Heatmap de la tabla de contingencia
plt.figure(figsize=(10, 6))
sns.heatmap(tabla_contingencia, annot=True, fmt='d', cmap='YlOrRd', cbar_kws={'label': 'Frecuencia'})
plt.title('Tabla de Contingencia: Tipo de Donante × Frecuencia de Donación', 
          fontsize=14, fontweight='bold')
plt.xlabel('Frecuencia de Donación', fontsize=12)
plt.ylabel('Tipo de Donante', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de barras agrupadas para mejor interpretación
tabla_contingencia.plot(kind='bar', figsize=(12, 6), width=0.8)
plt.title('Frecuencia de Donación por Tipo de Donante', fontsize=14, fontweight='bold')
plt.xlabel('Tipo de Donante', fontsize=12)
plt.ylabel('Número de Donadores', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.legend(title='Frecuencia de Donación', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

**📝 Pregunta 1.2:** Observa la tabla de contingencia y los gráficos. ¿Qué patrones preliminares notas?

*Escribe tu observación aquí:*

**RESPUESTA:**

[TU OBSERVACIÓN AQUÍ]

### 1.3 Ejecución de la Prueba Chi-Cuadrada (10 puntos)

In [ ]:
# TU CÓDIGO AQUÍ: Realiza la prueba de Chi-cuadrada
# Sugerencia: chi2_contingency(tabla_contingencia)

chi2_stat, p_value, dof, expected_freq = 

print("="*70)
print("🧪 RESULTADOS DE LA PRUEBA CHI-CUADRADA (χ²)")
print("="*70)
print(f"Estadístico χ²: {chi2_stat:.4f}")
print(f"Valor p: {p_value:.4f}")
print(f"Grados de libertad: {dof}")
print(f"Nivel de significancia (α): 0.05")
print("="*70)

# Mostrar frecuencias esperadas bajo H₀
print("\n📊 FRECUENCIAS ESPERADAS (bajo H₀ de independencia):")
expected_df = pd.DataFrame(
    expected_freq, 
    index=tabla_contingencia.index, 
    columns=tabla_contingencia.columns
)
print(expected_df.round(2))

In [ ]:
# Calcular residuos estandarizados para identificar celdas contribuyendo a χ²
residuos = (tabla_contingencia - expected_df) / np.sqrt(expected_df)

print("\n📏 RESIDUOS ESTANDARIZADOS (Contribución a χ²):")
print("Valores absolutos > 2 indican desviaciones importantes de independencia\n")
print(residuos.round(2))

# Visualizar residuos
plt.figure(figsize=(10, 6))
sns.heatmap(residuos, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            cbar_kws={'label': 'Residuo Estandarizado'})
plt.title('Residuos Estandarizados: Tipo × Frecuencia\n(Valores >2 o <-2 indican asociación)', 
          fontsize=13, fontweight='bold')
plt.xlabel('Frecuencia de Donación')
plt.ylabel('Tipo de Donante')
plt.tight_layout()
plt.show()

### 1.4 Decisión e Interpretación (5 puntos)

In [ ]:
# TU CÓDIGO AQUÍ: Lógica de decisión
alpha = 0.05

if p_value < alpha:
    decision = "Rechazamos H₀"
    conclusion = "Existe asociación estadísticamente significativa entre tipo de donante y frecuencia de donación"
else:
    decision = "No rechazamos H₀"
    conclusion = "No hay evidencia suficiente de asociación entre tipo de donante y frecuencia de donación"

print("\n📋 DECISIÓN ESTADÍSTICA:\n")
print(f"Decisión: {decision}")
print(f"\nConclusión: {conclusion}")

**📝 Pregunta 1.4:** Interpreta los resultados en contexto de negocio.

a) **Decisión:** ¿Rechazaste H₀? ¿Por qué?

b) **Residuos:** Observa el mapa de residuos. ¿Qué celdas tienen residuos > 2 o < -2? ¿Qué patrones específicos existen?

c) **Implicaciones estratégicas:** Si hay asociación, ¿qué estrategias de engagement diferentes debería implementar la Fundación para cada tipo de donador?

*Escribe tus respuestas aquí:*

**RESPUESTA:**

a) [TU RESPUESTA]

b) [TU RESPUESTA]

c) [TU RESPUESTA]

---

## Parte 2: ANOVA (Análisis de Varianza) (35 puntos)

### Contexto

**Pregunta de negocio:** ¿La satisfacción general (`satisfaccion_general`) varía según el canal de donación (`canal_donacion`)?

**Hipótesis intuitiva:**
- Canales con **interacción humana** (Presencial, Evento) podrían generar mayor satisfacción
- Canal **Online** es conveniente pero menos personal
- Canal **Teléfono** puede ser percibido como intrusivo

**¿Por qué ANOVA?** Comparar medias de variable continua (satisfaccion_general) entre >2 grupos categóricos (4 canales).

---

### 2.1 Planteamiento de Hipótesis (5 puntos)

**📝 Ejercicio 2.1:** Escribe las hipótesis del ANOVA.

**Hipótesis:**

- **H₀ (Nula):** [TU RESPUESTA - ¿Las medias de satisfacción son iguales entre todos los canales?]

- **H₁ (Alternativa):** [TU RESPUESTA - ¿Al menos una media es diferente?]

**Nivel de significancia:** α = 0.05

### 2.2 Exploración Descriptiva por Grupos (5 puntos)

In [ ]:
# TU CÓDIGO AQUÍ: Calcula estadísticas descriptivas por canal
# Sugerencia: df.groupby('canal_donacion')['satisfaccion_general'].agg(...)

print("📊 ESTADÍSTICAS DESCRIPTIVAS: Satisfacción General por Canal de Donación\n")

stats_canal = 

print(stats_canal)

In [ ]:
# Visualización 1: Boxplot
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='canal_donacion', y='satisfaccion_general', palette='Set2')
sns.stripplot(data=df, x='canal_donacion', y='satisfaccion_general', 
              color='black', alpha=0.2, size=3)
plt.title('Comparación de Satisfacción General por Canal de Donación', 
          fontsize=14, fontweight='bold')
plt.xlabel('Canal de Donación', fontsize=12)
plt.ylabel('Satisfacción General (1-10)', fontsize=12)
plt.axhline(df['satisfaccion_general'].mean(), color='red', linestyle='--', 
            linewidth=2, label=f'Media global: {df["satisfaccion_general"].mean():.2f}')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Visualización 2: Violin plot (muestra distribución)
plt.figure(figsize=(12, 6))
sns.violinplot(data=df, x='canal_donacion', y='satisfaccion_general', palette='muted')
plt.title('Distribución de Satisfacción por Canal de Donación (Violin Plot)', 
          fontsize=14, fontweight='bold')
plt.xlabel('Canal de Donación', fontsize=12)
plt.ylabel('Satisfacción General (1-10)', fontsize=12)
plt.tight_layout()
plt.show()

**📝 Pregunta 2.2:** Observa los gráficos y las estadísticas.

a) ¿Qué canal tiene la media de satisfacción más alta? ¿Y la más baja?

b) ¿Hay diferencias visuales notables entre las distribuciones?

c) ¿Algún canal tiene mayor variabilidad (mayor desviación estándar)?

*Escribe tus observaciones aquí:*

**RESPUESTA:**

a) [TU RESPUESTA]

b) [TU RESPUESTA]

c) [TU RESPUESTA]

### 2.3 Verificación de Supuestos (OPCIONAL - 5 puntos BONUS)

ANOVA asume:
1. **Normalidad** en cada grupo
2. **Homogeneidad de varianzas** (homocedasticidad)

Para esta práctica, asumiremos que se cumplen. Si quieres ganar puntos bonus, realiza los tests:

In [ ]:
# BONUS: Test de Levene para homogeneidad de varianzas
# TU CÓDIGO AQUÍ (opcional)

# from scipy.stats import levene
# grupos = [df[df['canal_donacion'] == canal]['satisfaccion_general'] for canal in df['canal_donacion'].unique()]
# levene_stat, levene_p = levene(*grupos)
# print(f"Test de Levene: W = {levene_stat:.4f}, p = {levene_p:.4f}")
# if levene_p > 0.05:
#     print("✓ Supuesto de homogeneidad de varianzas se cumple")
# else:
#     print("✗ Varianzas heterogéneas - considerar Welch ANOVA")

### 2.4 Ejecución del ANOVA (10 puntos)

In [ ]:
# TU CÓDIGO AQUÍ: Separa los grupos y ejecuta ANOVA
# Sugerencia: Crea listas con datos de cada canal, luego usa stats.f_oneway()

grupo_online = 
grupo_evento = 
grupo_presencial = 
grupo_telefono = 

# Ejecutar ANOVA
f_stat, p_value_anova = 

print("="*70)
print("🧪 RESULTADOS DEL ANOVA (ONE-WAY)")
print("="*70)
print(f"Estadístico F: {f_stat:.4f}")
print(f"Valor p: {p_value_anova:.4f}")
print(f"Nivel de significancia (α): 0.05")
print("="*70)

# Calcular tamaño del efecto (eta cuadrado)
# η² = SSbetween / SStotal
grand_mean = df['satisfaccion_general'].mean()
ss_between = sum([len(df[df['canal_donacion'] == canal]) * 
                  (df[df['canal_donacion'] == canal]['satisfaccion_general'].mean() - grand_mean)**2 
                  for canal in df['canal_donacion'].unique()])
ss_total = sum((df['satisfaccion_general'] - grand_mean)**2)
eta_squared = ss_between / ss_total

print(f"\n📏 Tamaño del efecto (η²): {eta_squared:.4f}")
print("Interpretación: ", end="")
if eta_squared < 0.01:
    print("Efecto pequeño (<0.01)")
elif eta_squared < 0.06:
    print("Efecto pequeño-mediano (0.01-0.06)")
elif eta_squared < 0.14:
    print("Efecto mediano (0.06-0.14)")
else:
    print("Efecto grande (>0.14)")

### 2.5 Prueba Post-Hoc (Tukey HSD) (10 puntos)

Si rechazaste H₀, realiza prueba post-hoc para identificar qué pares de canales son diferentes.

In [ ]:
# TU CÓDIGO AQUÍ: Prueba de Tukey HSD
# Sugerencia: pairwise_tukeyhsd(endog=df['satisfaccion_general'], groups=df['canal_donacion'], alpha=0.05)

tukey_result = 

print("🔍 RESULTADOS DE LA PRUEBA POST-HOC (TUKEY HSD)")
print("="*70)
print(tukey_result)
print("="*70)
print("\nInterpretación:")
print("- 'reject = True' indica diferencia significativa entre ese par de grupos")
print("- 'meandiff' muestra la diferencia de medias")
print("- Si el intervalo de confianza NO contiene 0, la diferencia es significativa")

In [ ]:
# Visualizar resultados de Tukey
fig, ax = plt.subplots(figsize=(10, 6))
tukey_result.plot_simultaneous(ax=ax)
plt.title('Intervalos de Confianza de Tukey HSD (95%)\nGrupos que no se traslapan son significativamente diferentes', 
          fontsize=13, fontweight='bold')
plt.xlabel('Satisfacción General (1-10)')
plt.tight_layout()
plt.show()

**📝 Pregunta 2.5:** Interpreta los resultados del ANOVA y Tukey.

a) **Decisión ANOVA:** ¿Rechazaste H₀? ¿Qué significa esto?

b) **Diferencias específicas:** Según Tukey, ¿qué pares de canales son significativamente diferentes? Lista todos los pares con p < 0.05.

c) **Ranking de canales:** Ordena los canales de mayor a menor satisfacción.

d) **Recomendaciones estratégicas:** 
   - ¿Qué canal(es) debería priorizar la Fundación para maximizar satisfacción?
   - ¿Qué mejoras específicas recomiendas para el(los) canal(es) con menor satisfacción?

*Escribe tus respuestas aquí:*

**RESPUESTA:**

a) [TU RESPUESTA]

b) [TU RESPUESTA]

c) [TU RESPUESTA]

d) [TU RESPUESTA]

---

## Parte 3: Regresión Lineal Simple (40 puntos)

### Contexto

**Pregunta de negocio:** ¿Los años como donador (`años_como_donante`) predicen el nivel de compromiso futuro (`compromiso_futuro`)?

**Hipótesis intuitiva:**
- **Antigüedad** debería correlacionar con **lealtad** (compromiso futuro)
- Donadores con más años de relación están más "invertidos" emocionalmente
- Sin embargo, podría haber efecto techo (después de X años, no aumenta más)

**¿Por qué Regresión?** Queremos **predecir** una variable continua (compromiso_futuro) usando otra variable continua (años_como_donante).

---

### 3.1 Análisis Exploratorio (5 puntos)

In [ ]:
# TU CÓDIGO AQUÍ: Scatter plot de años vs compromiso

plt.figure(figsize=(10, 6))
plt.scatter(df['años_como_donante'], df['compromiso_futuro'], alpha=0.5, edgecolors='k')
plt.title('Relación entre Años como Donador y Compromiso Futuro', fontsize=14, fontweight='bold')
plt.xlabel('Años como Donante', fontsize=12)
plt.ylabel('Compromiso Futuro (1-10)', fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

**📝 Pregunta 3.1:** Observa el scatter plot.

a) ¿Parece haber una relación lineal entre ambas variables?

b) ¿La relación es positiva o negativa?

c) ¿La relación parece fuerte, moderada o débil?

*Escribe tus observaciones aquí:*

**RESPUESTA:**

a) [TU RESPUESTA]

b) [TU RESPUESTA]

c) [TU RESPUESTA]

### 3.2 Correlación de Pearson (5 puntos)

In [ ]:
# TU CÓDIGO AQUÍ: Calcula correlación de Pearson
# Sugerencia: stats.pearsonr(df['años_como_donante'], df['compromiso_futuro'])

r_pearson, p_value_corr = 

print("="*70)
print("📊 CORRELACIÓN DE PEARSON")
print("="*70)
print(f"Coeficiente de correlación (r): {r_pearson:.4f}")
print(f"Valor p: {p_value_corr:.4f}")
print("="*70)

# Interpretación de magnitud
print("\nInterpretación de magnitud (Cohen, 1988):")
if abs(r_pearson) < 0.10:
    print("  Correlación trivial/nula (<0.10)")
elif abs(r_pearson) < 0.30:
    print("  Correlación pequeña (0.10-0.30)")
elif abs(r_pearson) < 0.50:
    print("  Correlación mediana (0.30-0.50)")
else:
    print("  Correlación grande (>0.50)")

# Interpretación de dirección
if r_pearson > 0:
    print(f"\nDirección: Correlación POSITIVA")
    print(f"  → A mayor {' años_como_donante'}, mayor {'compromiso_futuro'}")
elif r_pearson < 0:
    print(f"\nDirección: Correlación NEGATIVA")
    print(f"  → A mayor {'años_como_donante'}, menor {'compromiso_futuro'}")
else:
    print(f"\nDirección: Sin correlación")

**📝 Pregunta 3.2:** Interpreta la correlación.

a) ¿La correlación es estadísticamente significativa (p < 0.05)?

b) ¿Qué tan fuerte es la relación lineal?

c) ¿Qué significa este valor de r en términos prácticos para la Fundación?

*Escribe tus interpretaciones aquí:*

**RESPUESTA:**

a) [TU RESPUESTA]

b) [TU RESPUESTA]

c) [TU RESPUESTA]

### 3.3 Creación del Modelo de Regresión Lineal (10 puntos)

In [ ]:
# TU CÓDIGO AQUÍ: Ajusta el modelo de regresión lineal
# Sugerencia: Usa statsmodels (sm.OLS)

# Preparar datos
X = df['años_como_donante']
y = df['compromiso_futuro']

# Añadir constante (intercepto)
X_const = sm.add_constant(X)

# Ajustar modelo
modelo = 

# Mostrar resumen completo
print("="*70)
print("📊 RESUMEN DEL MODELO DE REGRESIÓN LINEAL")
print("="*70)
print(modelo.summary())

### 3.4 Extracción e Interpretación de Parámetros (10 puntos)

In [ ]:
# TU CÓDIGO AQUÍ: Extrae los parámetros del modelo

intercepto = modelo.params['const']
pendiente = modelo.params['años_como_donante']
r_cuadrado = modelo.rsquared
r_cuadrado_adj = modelo.rsquared_adj

print("="*70)
print("📏 PARÁMETROS DEL MODELO")
print("="*70)
print(f"Intercepto (β₀): {intercepto:.4f}")
print(f"Pendiente (β₁): {pendiente:.4f}")
print(f"R² (R-cuadrado): {r_cuadrado:.4f}")
print(f"R² ajustado: {r_cuadrado_adj:.4f}")
print("="*70)

print(f"\n📐 ECUACIÓN DEL MODELO:\n")
print(f"   compromiso_futuro = {intercepto:.3f} + ({pendiente:.4f}) × años_como_donante")
print("="*70)

print(f"\n📊 INTERPRETACIÓN DE LOS COEFICIENTES:\n")
print(f"1. Intercepto (β₀ = {intercepto:.3f}):")
print(f"   → El compromiso futuro esperado cuando años_como_donante = 0")
print(f"   → Representa el nivel base de compromiso de un donador nuevo\n")

print(f"2. Pendiente (β₁ = {pendiente:.4f}):")
if pendiente > 0:
    print(f"   → Por cada año adicional como donante, el compromiso futuro AUMENTA {pendiente:.4f} puntos")
else:
    print(f"   → Por cada año adicional como donante, el compromiso futuro DISMINUYE {abs(pendiente):.4f} puntos")

print(f"\n3. R² ({r_cuadrado:.4f}):")
print(f"   → El modelo explica el {r_cuadrado*100:.2f}% de la variabilidad en compromiso futuro")
print(f"   → El {(1-r_cuadrado)*100:.2f}% restante se debe a otros factores no incluidos en el modelo")

**📝 Pregunta 3.4:** Analiza los parámetros del modelo.

a) **Intercepto:** ¿Tiene sentido práctico el valor del intercepto en este contexto?

b) **Pendiente:** ¿Es positiva o negativa? ¿Qué implica esto? ¿El efecto es sustancial o marginal?

c) **R²:** ¿El modelo tiene buen ajuste? ¿Qué porcentaje de varianza queda sin explicar? ¿Qué otros factores podrían influir en el compromiso futuro?

*Escribe tus análisis aquí:*

**RESPUESTA:**

a) [TU RESPUESTA]

b) [TU RESPUESTA]

c) [TU RESPUESTA]

### 3.5 Visualización del Modelo (5 puntos)

In [ ]:
# TU CÓDIGO AQUÍ: Grafica datos con línea de regresión

plt.figure(figsize=(12, 7))

# Scatter de datos observados
plt.scatter(df['años_como_donante'], df['compromiso_futuro'], 
            alpha=0.5, edgecolors='k', label='Datos observados', s=50)

# Línea de regresión
x_line = np.linspace(df['años_como_donante'].min(), df['años_como_donante'].max(), 100)
y_line = intercepto + pendiente * x_line
plt.plot(x_line, y_line, 'r-', linewidth=3, 
         label=f'Regresión: ŷ = {intercepto:.2f} + {pendiente:.4f}x')

# Banda de confianza (95%)
# TU CÓDIGO AQUÍ (opcional para mostrar intervalo de confianza)

plt.title('Modelo de Regresión Lineal: Años como Donante → Compromiso Futuro', 
          fontsize=14, fontweight='bold')
plt.xlabel('Años como Donante', fontsize=12)
plt.ylabel('Compromiso Futuro (1-10)', fontsize=12)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)

# Añadir anotación con R²
plt.text(0.05, 0.95, f'R² = {r_cuadrado:.4f}', 
         transform=plt.gca().transAxes, fontsize=13, fontweight='bold',
         verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

### 3.6 Análisis de Residuos (5 puntos)

Los residuos deben ser aleatorios y sin patrones para validar el modelo lineal.

In [ ]:
# Calcular residuos
residuos = modelo.resid
valores_predichos = modelo.fittedvalues

# Gráficos de diagnóstico
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Residuos vs Valores Predichos
axes[0].scatter(valores_predichos, residuos, alpha=0.5, edgecolors='k')
axes[0].axhline(0, color='red', linestyle='--', linewidth=2)
axes[0].set_title('Residuos vs Valores Predichos', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Valores Predichos (ŷ)')
axes[0].set_ylabel('Residuos (e = y - ŷ)')
axes[0].grid(alpha=0.3)

# 2. QQ-Plot (normalidad de residuos)
from scipy.stats import probplot
probplot(residuos, dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot: Normalidad de Residuos', fontsize=12, fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 INTERPRETACIÓN DE GRÁFICOS DE RESIDUOS:\n")
print("1. Residuos vs Predichos:")
print("   ✓ Si los puntos están dispersos aleatoriamente alrededor de 0 → modelo adecuado")
print("   ✗ Si hay patrón (curva, embudo) → modelo lineal no es apropiado\n")
print("2. Q-Q Plot:")
print("   ✓ Si los puntos siguen la línea diagonal → residuos son normales")
print("   ✗ Si se desvían de la línea → residuos no son normales")

### 3.7 Predicciones con el Modelo (5 puntos)

In [ ]:
# TU CÓDIGO AQUÍ: Realiza predicciones para diferentes valores de años

años_prediccion = [0, 2, 5, 10, 15, 20]

print("="*70)
print("🔮 PREDICCIONES DEL MODELO")
print("="*70)

for años in años_prediccion:
    # Calcular predicción usando la ecuación del modelo
    compromiso_pred = 
    
    # Crear DataFrame para predict (necesita matriz con constante)
    X_new = pd.DataFrame({'const': [1], 'años_como_donante': [años]})
    
    # Predicción con intervalo de confianza
    pred_interval = modelo.get_prediction(X_new).summary_frame(alpha=0.05)
    
    print(f"\nAños: {años:2d} → Compromiso predicho: {compromiso_pred:.2f}")
    print(f"             IC 95%: [{pred_interval['mean_ci_lower'].values[0]:.2f}, {pred_interval['mean_ci_upper'].values[0]:.2f}]")

print("\n" + "="*70)

**📝 Pregunta 3.7:** Analiza las predicciones.

a) ¿Las predicciones son razonables (dentro de 1-10)?

b) ¿Cómo cambia la predicción de compromiso entre un donador nuevo (0 años) y uno con 10 años de antigüedad?

c) **Aplicación práctica:** Si la Fundación quiere aumentar el compromiso futuro promedio de 7.0 a 8.0, ¿cuántos años de relación necesitarían en promedio según el modelo?

*Escribe tus análisis aquí:*

**RESPUESTA:**

a) [TU RESPUESTA]

b) [TU RESPUESTA]

c) [TU RESPUESTA - Hint: Despeja años en la ecuación cuando compromiso = 8.0]

---

## Parte 4: Reflexión MEAL Comparativa - Semanas 1 y 2 (20 puntos)

### 📝 Estructura Requerida: MEAL Integradora

Esta reflexión debe **sintetizar hallazgos de AMBAS semanas** de análisis estadístico.

---

### 📊 Requisitos de Contenido

| Componente | Requisitos | Palabras | Puntos |
|------------|-----------|----------|--------|
| **M - Main Idea** | Tesis integradora sobre retención de donadores basada en ambas semanas | 50-75 | 5 |
| **E - Evidence** | Datos específicos de: descriptivos, t-tests, chi², ANOVA, regresión | 100-125 | 5 |
| **A - Analysis** | Interpretación con ≥2 citas APA 7, discusión de causalidad/correlación | 100-125 | 5 |
| **L - Link** | Implicaciones para estrategia de retención, aprendizajes del proceso | 50-75 | 5 |
| **TOTAL** | | **300-400** | **20** |

---

### 🎯 Preguntas Guía (No respondas como lista, integra en MEAL)

**Integración de hallazgos:**
1. ¿Qué factores consistentemente predicen/se asocian con retención y compromiso?
2. ¿Cómo se complementan los resultados de técnicas univariadas (Semana 1) y multivariadas (Semana 2)?
3. ¿Qué contradicciones o resultados inesperados encontraste?

**Comparación metodológica:**
4. ¿Cuándo usarías chi-cuadrada vs t-test vs ANOVA vs regresión?
5. ¿Qué ventajas tiene cada técnica para responder diferentes tipos de preguntas?

**Estrategia de retención:**
6. Si fueras Director de Desarrollo de Fundación Esperanza, ¿qué 3 iniciativas priorizarías basándote en TODOS los análisis?
7. ¿Qué segmentos de donadores requieren estrategias diferenciadas?

**Limitaciones y siguientes pasos:**
8. ¿Qué limitaciones tienen estos análisis en conjunto?
9. ¿Qué análisis adicionales recomendarías (ej: regresión múltiple, análisis de supervivencia)?

---

### 💡 Plantilla de Estructura

```markdown
**[M - Main Idea - 50-75 palabras]**
[Tesis que sintetiza hallazgos sobre qué factores predicen retención de donadores, 
integrando resultados de Semana 1 (descriptivos, t-tests) y Semana 2 (chi², ANOVA, regresión)]

**[E - Evidence - 100-125 palabras]**
[Semana 1: Resultados de satisfacción, voluntariado, confianza]
[Semana 2: Resultados de chi-cuadrada, ANOVA canales, regresión años-compromiso]
[Comparación cuantitativa de tamaños de efecto y significancias]

**[A - Analysis - 100-125 palabras]**
[Interpretación de patrones integrados]
[Conexión con literatura de donor retention - Cita 1 (Autor, año)]
[Análisis de causalidad vs correlación - Cita 2 (Autor, año)]
[Discusión de implicaciones teóricas]

**[L - Link - 50-75 palabras]**
[3 recomendaciones estratégicas priorizadas para Fundación Esperanza]
[Aprendizajes personales sobre proceso de análisis estadístico completo]
[Próximos pasos en investigación o implementación]
```

---

### 📚 Referencias Sugeridas (Cita al menos 2)

**Sobre Donor Retention:**

Sargeant, A. (2001). Relationship fundraising: How to keep donors loyal. *Nonprofit Management and Leadership, 12*(2), 177-192. https://doi.org/10.1002/nml.12203

> Cuándo citar: Para discutir teoría de relationship fundraising y factores de retención (satisfacción, confianza, engagement)

Bekkers, R., & Wiepking, P. (2011). A literature review of empirical studies of philanthropy: Eight mechanisms that drive charitable giving. *Nonprofit and Voluntary Sector Quarterly, 40*(5), 924-973. https://doi.org/10.1177/0899764010380927

> Cuándo citar: Para contextualizar mecanismos de donación (awareness, solicitation, costs/benefits, altruism, reputation, psychological benefits, values, efficacy)

**Sobre Engagement:**

Waters, R. D. (2008). Applying relationship management theory to the fundraising process for individual donors. *Journal of Communication Management, 12*(1), 73-87. https://doi.org/10.1108/13632540810854244

> Cuándo citar: Para discutir importancia de engagement y comunicación en retención

**Sobre Métodos Estadísticos:**

Field, A. (2013). *Discovering statistics using IBM SPSS Statistics* (4th ed.). Sage Publications.

> Cuándo citar: Para discutir interpretación de tamaños de efecto o comparación entre técnicas estadísticas

---

### ✍️ ESCRIBE TU REFLEXIÓN AQUÍ

**[M - Main Idea]**

[Escribe tu tesis integradora aquí - 50-75 palabras]

---

**[E - Evidence]**

[Escribe tu evidencia de ambas semanas aquí - 100-125 palabras]

---

**[A - Analysis]**

[Escribe tu análisis con ≥2 citas APA 7 aquí - 100-125 palabras]

---

**[L - Link]**

[Escribe tus recomendaciones y aprendizajes aquí - 50-75 palabras]

---

**Número total de palabras:** [Cuenta tus palabras - debe ser 300-400]

---

---

## 📊 Resumen de Tareas Completadas

**Marca con ✅ las tareas completadas:**

- [ ] Parte 0: Configuración y carga (5 pts)
- [ ] Parte 1: Prueba Chi-Cuadrada (25 pts)
  - [ ] 1.1 Hipótesis
  - [ ] 1.2 Tabla de contingencia
  - [ ] 1.3 Ejecución de prueba
  - [ ] 1.4 Interpretación
- [ ] Parte 2: ANOVA (35 pts)
  - [ ] 2.1 Hipótesis
  - [ ] 2.2 Exploración descriptiva
  - [ ] 2.3 Verificación de supuestos (OPCIONAL BONUS)
  - [ ] 2.4 Ejecución de ANOVA
  - [ ] 2.5 Prueba Post-Hoc (Tukey)
- [ ] Parte 3: Regresión Lineal (40 pts)
  - [ ] 3.1 Análisis exploratorio
  - [ ] 3.2 Correlación de Pearson
  - [ ] 3.3 Creación del modelo
  - [ ] 3.4 Interpretación de parámetros
  - [ ] 3.5 Visualización del modelo
  - [ ] 3.6 Análisis de residuos
  - [ ] 3.7 Predicciones
- [ ] Parte 4: Reflexión MEAL Comparativa (20 pts)

**Puntuación total:** ___ / 100 puntos (+ bonus)

---

## 🎓 Autoevaluación Final

**1. Comparación de Semanas:**

¿Qué técnica te resultó más útil para responder preguntas de negocio: las de Semana 1 (descriptivos, t-tests) o las de Semana 2 (chi², ANOVA, regresión)? ¿Por qué?

[TU RESPUESTA]

**2. Integración de Hallazgos:**

¿Cómo cambió tu comprensión del problema de retención al usar múltiples técnicas estadísticas en vez de solo una?

[TU RESPUESTA]

**3. Aplicación Práctica:**

Si tuvieras que presentar un resumen ejecutivo de 2 minutos a la Junta Directiva de Fundación Esperanza, ¿cuáles serían tus 3 mensajes clave basados en TODOS los análisis?

[TU RESPUESTA]

**4. Aprendizaje Metodológico:**

¿Qué concepto estadístico que antes no entendías ahora comprendes mejor después de estas 2 semanas?

[TU RESPUESTA]

---

## 🚀 Próximos Pasos

**Felicitaciones por completar las prácticas de estadística aplicada!** 🎉

**Para seguir aprendiendo:**

1. **Profundiza en regresión:**
   - Regresión lineal múltiple (múltiples predictores)
   - Regresión logística (para predecir donara_proximo_año directamente)

2. **Explora técnicas avanzadas:**
   - Análisis de supervivencia (tiempo hasta abandono de donador)
   - Análisis de clustering (segmentación de donadores)
   - Series de tiempo (predicción de donaciones futuras)

3. **Aplica a tu reto final:**
   - Usa estas mismas técnicas en el dataset de tu reto
   - Adapta la estructura MEAL a tu contexto específico

4. **Consulta recursos adicionales:**
   - CODEBOOK completo: `CODEBOOK_fundacion_esperanza.md`
   - Notebooks de workshops oficiales (Semana 1 y 2)
   - Documentación de scipy.stats y statsmodels

---

**¡Excelente trabajo!** Has completado un análisis estadístico completo de principio a fin, aplicando 6 técnicas diferentes. Estas habilidades son directamente transferibles a problemas reales de negocios, investigación y política pública. 🎓📊✨